In [77]:
import numpy as np


In [83]:
input = """#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^"""

In [79]:
input = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

In [80]:
def get_direction(move):
    if move == "<":
        return (0,-1)
    elif move == "v":
        return (1,0)
    elif move == ">":
        return (0,1)
    else:
        return (-1,0)


In [81]:
def parse_grid_line(line):
    line_as_list = [""] * len(line) * 2
    for i, tile in enumerate(line):
        if tile == "#" or tile == "..":
            line_as_list[(i*2)] = tile
            line_as_list[(i*2) + 1] = tile
        elif tile == "O":
            line_as_list[(i*2)] = "["
            line_as_list[(i*2) + 1] = "]"
        elif tile == "@":
            line_as_list[(i*2)] = "@"
            line_as_list[(i*2) + 1] = "."
        elif tile == ".":
            line_as_list[(i*2)] = tile
            line_as_list[(i*2) + 1] = tile
    return line_as_list

def parse_input(input):
    grid_part, move_sequence_part = input.split("\n\n")
    grid = np.array([parse_grid_line(line) for line in grid_part.splitlines()])
    
    move_sequence = list(move_sequence_part.replace("\n", "").strip())
    return grid, move_sequence

In [106]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))
grid, move_sequence = parse_input(input)
print(grid)
current_robot_position = np.where(grid == '@')

def get_update_vector(current_position, direction_vector):
    target_tile_position = current_position[0] + direction_vector[0], current_position[1] + direction_vector[1]
    target_tile = grid[target_tile_position[0], target_tile_position[1]]
    if target_tile == ".":
        return direction_vector
    elif target_tile == "#":
        return [0,0]
    elif target_tile == "O":
        update_vector = get_update_vector(target_tile_position, direction_vector)
        grid[target_tile_position[0], target_tile_position[1]] = "."
        grid[target_tile_position[0] + update_vector[0], target_tile_position[1] + update_vector[1]] = target_tile
        return update_vector

def update_grid(current_position, target_tile_position):
    current_tile = grid[current_position[0], current_position[1]]
    grid[current_position[0], current_position[1]] = "."
    grid[target_tile_position[0], target_tile_position[1]] = current_tile

def box_will_not_hit_wall_check(current_position, direction_vector):
    target_tile_position = [current_position[0] + direction_vector[0], current_position[1] + direction_vector[1]]
    target_tile = grid[target_tile_position[0], target_tile_position[1]]

    if target_tile == ".":
        update_grid(current_position, target_tile_position)
        return True
    elif target_tile == "#":
        return False
    elif target_tile == "[" or target_tile == "]":
        res = box_will_not_hit_wall_check(target_tile_position, direction_vector)
        if res:
            update_grid(current_position, target_tile_position)
        return res
    
def get_update_y_positions(accu, current_pos, direction_vector):
    target_tile_position = [current_pos[0] + direction_vector[0], current_pos[1] + direction_vector[1]]
    target_tile = grid[target_tile_position[0], target_tile_position[1]]
    target_tile_adjacent_tile_position = []
    if target_tile == "[":
        target_tile_adjacent_tile_position = target_tile_position[0], target_tile_position[1] + 1
    elif target_tile == "]":
        target_tile_adjacent_tile_position = target_tile_position[0], target_tile_position[1] - 1
    target_tiles_adjacent_tiles_target_tile_position = [target_tile_adjacent_tile_position[0] + direction_vector[0], target_tile_adjacent_tile_position[1] + direction_vector[1]]
    target_tiles_target_tile_position = target_tile_position[0]+ direction_vector[0], target_tile_position[1] + direction_vector[1]
    target_tiles_adjacent_tiles_target_tile = grid[target_tiles_adjacent_tiles_target_tile_position[0], target_tiles_adjacent_tiles_target_tile_position[1]]
    target_tiles_target_tile = grid[target_tiles_target_tile_position[0], target_tiles_target_tile_position[1]]

    if target_tiles_adjacent_tiles_target_tile == "." and target_tiles_target_tile == ".":
        return [target_tiles_adjacent_tiles_target_tile_position, target_tiles_target_tile_position]
    elif (target_tiles_adjacent_tiles_target_tile == "#" or target_tiles_target_tile == "#"):
        return []
    elif (target_tiles_adjacent_tiles_target_tile == "[" or target_tiles_adjacent_tiles_target_tile == "]") or (target_tiles_target_tile == "[" or target_tiles_target_tile  == "]"):
        target_tile_target_tile_result = get_update_y_positions(accu, target_tile_position, direction_vector)
        target_tiles_adjacent_tiles_result = get_update_y_positions(accu, target_tile_adjacent_tile_position, direction_vector)
        if target_tile_target_tile_result > 0 and target_tiles_adjacent_tiles_result > 0:
            accu += target_tile_target_tile_result + target_tiles_adjacent_tiles_result
        else:
            accu = []

for move in move_sequence:
    print(move)
    direction_vector = get_direction(move)
    target_tile_position = [current_robot_position[0] + direction_vector[0], current_robot_position[1] + direction_vector[1]]
    target_tile = grid[target_tile_position[0], target_tile_position[1]]

    if target_tile == ".":
        update_grid(current_robot_position, target_tile_position)
        current_robot_position = target_tile_position
    elif target_tile == "[" or target_tile == "]":
        if abs(direction_vector[0]) == 1: # either up or down
            update_positions = []
            update_positions = get_update_y_positions(update_positions, current_robot_position, direction_vector)
            while update_positions:
                update_y_position = update_positions.pop()
                update_grid(update_y_position, [update_y_position[0] + direction_vector[0], update_y_position[1] + direction_vector[1]])
        else: 
            result = box_will_not_hit_wall_check(target_tile_position, direction_vector)
            if result:
                update_grid(current_robot_position, target_tile_position)
                current_robot_position = target_tile_position
    print(grid)
    
def calculate_total_gps(grid):
    total_gps = 0
    all_box_positions = np.where(grid == 'O')
    for top_edge_distance in all_box_positions[0]:
        total_gps += top_edge_distance * 100
    for left_edge_distance in all_box_positions[1]:
        total_gps += left_edge_distance
    return total_gps

print(calculate_total_gps(grid))

[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' '#' '.' '.' '.' '.' '.' '.' '#' '#' '.' '.' '#' '#']
 ['#' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '#']
 ['#' '#' '.' '.' '.' '.' '[' ']' '[' ']' '@' '.' '#' '#']
 ['#' '#' '.' '.' '.' '.' '[' ']' '.' '.' '.' '.' '#' '#']
 ['#' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
<
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' '#' '.' '.' '.' '.' '.' '.' '#' '#' '.' '.' '#' '#']
 ['#' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '#']
 ['#' '#' '.' '.' '.' '[' ']' '[' ']' '@' '.' '.' '#' '#']
 ['#' '#' '.' '.' '.' '.' '[' ']' '.' '.' '.' '.' '#' '#']
 ['#' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
v
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' '#' '.' '.' '.' '.' '.' '.' '#' '#' '.' '.' '#' '#']
 ['#' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' 

TypeError: '>' not supported between instances of 'list' and 'int'